# TD learning; the agent that can see

# Remember to check the number of samples for alpha and beta

now I'm gonna add numbers to the model. 

In [4]:
import os
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import statsmodels.api as sm
import scipy.stats as stats
import seaborn as sns
from sklearn.metrics import confusion_matrix
np.random.seed(42)
from joblib import Parallel, delayed
import matplotlib.tri as tri
import matplotlib.colors as mcolors
from scipy.interpolate import griddata
from scipy.interpolate import RBFInterpolator
import matplotlib.ticker as mticker
import itertools
from sklearn.metrics import r2_score
import optuna

# important directories

In [5]:
folder_path = 'data_risk_added'


output_dir_model_evaluation = "13_RL_agent_TDlearn_output"
os.makedirs(output_dir_model_evaluation, exist_ok=True)

output_dir_plots = os.path.join(output_dir_model_evaluation, "plots")
os.makedirs(output_dir_plots, exist_ok=True)

output_dir_model_behavior = os.path.join(output_dir_model_evaluation, "model_behavior")
os.makedirs(output_dir_model_behavior, exist_ok=True)

In [6]:

dataframes = [pd.read_excel(os.path.join(folder_path, file)) for file in os.listdir(folder_path) if file.endswith('.xlsx')]


n_participant = len(dataframes)
print(f"there are {n_participant} participants.")


dataframes[0]

there are 35 participants.


,arrowRT,distribution,interTrialInterval,outcome,myCard,yourCard,spaceRT,totalReward,trialIndex,trialType,choice,block,timeoutRepeat,is_within_IQR,risk
0,2609,uniform,789,lose,4,2,335,9.5,0,response,arrowdown,1,0,1,0.375
1,597,uniform,853,win,9,4,407,10,1,response,arrowup,1,0,1,0.000
2,188,uniform,904,win,4,7,504,10.5,2,response,arrowdown,1,0,1,0.375
3,423,uniform,916,win,2,4,434,11,3,response,arrowdown,1,0,1,0.125
4,549,uniform,806,win,5,7,287,11.5,4,response,arrowdown,1,0,1,0.500
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
272,761,high,913,win,7,8,382,80,125,response,arrowdown,4,0,1,0.447
273,596,low,921,win,4,3,318,80.5,83,response,arrowup,4,0,1,0.385
274,414,low,950,win,2,7,335,81,77,response,arrowdown,4,0,1,0.243
275,1371,uniform,842,win,6,4,615,81.5,35,response,arrowup,4,0,1,0.375


### I want to make participant file name for the model_evaluation.csv and that is I'm gonna take each data name task_data_07_11_2024_17_23_43.xlsx and extract "07_11_2024_17_23_43" and this should be the participant name in the dataset. 

In [7]:
participants = [os.path.splitext(file)[0].replace("task_data_", "")
    for file in os.listdir(folder_path) if file.endswith('.xlsx')]

# policy initilization for the model
now I need to find the prior policy amounts. for that I am going to put the percentage of downarrow and up arrow for each distribution.

In [8]:
df_combined = pd.concat(dataframes, ignore_index=True)

df_combined = df_combined[df_combined['outcome'].str.lower() != 'na'].reset_index(drop=True)
 

desired_order = ["uniform", "low", "high"]  


cards_sorted = sorted(df_combined["myCard"].unique())
dist_sorted = [d for d in desired_order if d in df_combined["distribution"].unique()]
choice_sorted = sorted(df_combined["choice"].unique())


card_idx = {card: i for i, card in enumerate(cards_sorted)}
dist_idx = {dist: i for i, dist in enumerate(dist_sorted)}
choice_idx = {choice: i for i, choice in enumerate(choice_sorted)}


matrix_3d = np.zeros((len(cards_sorted), len(dist_sorted), len(choice_sorted)))


for _, row in df_combined.iterrows():
    i = card_idx[row["myCard"]]-1
    j = dist_idx[row["distribution"]]
    k = choice_idx[row["choice"]]
    matrix_3d[i, j, k] += 1  


total_per_card_dist = matrix_3d.sum(axis=2, keepdims=True)

# compute percentages, avoiding division by zero
with np.errstate(divide='ignore', invalid='ignore'):
    percentage_matrix = np.divide(matrix_3d, total_per_card_dist, where=total_per_card_dist != 0)

# convert to a DataFrame for easy visualization
percentage_list = []
for i, card in enumerate(cards_sorted):
    for j, dist in enumerate(dist_sorted):
        for k, choice in enumerate(choice_sorted):
            percentage_list.append({
                "myCard": card,
                "distribution": dist,  # Now follows "uniform", "low", "high" order
                "choice": choice,
                "percentage": percentage_matrix[i, j, k]
            })

df_percentages = pd.DataFrame(percentage_list)
df_percentages

,myCard,distribution,choice,percentage
0,1,uniform,arrowdown,0.965217
1,1,uniform,arrowup,0.034783
2,1,low,arrowdown,0.922399
3,1,low,arrowup,0.077601
4,1,high,arrowdown,0.958042
5,1,high,arrowup,0.041958
6,2,uniform,arrowdown,0.967836
7,2,uniform,arrowup,0.032164
8,2,low,arrowdown,0.748479
9,2,low,arrowup,0.251521


In [9]:
np.shape(percentage_matrix)

(9, 3, 2)

In [10]:
actions = { "arrowdown": 0, "arrowup": 1}
distributions_map = { "uniform": 0, "low": 1,  "high": 2}
card_numbers = list(range(1, 10))

policy_table = percentage_matrix 

Q_table_init = np.random.normal(0, 0.1, (len(card_numbers), len(distributions_map), len(actions)))
# having a q-table based on the policies
Q_table_init = policy_table * np.mean(Q_table_init) 
Q_table = Q_table_init.copy()

#############################################################################################
# having a q-table that starts with 0! this was not a good initilization so i changed it.
# Q_table = np.zeros((len(distributions_map), len(actions)))  # 3 distributions × 2 actions
#############################################################################################

print("policy: \n",np.shape(policy_table))
print("\n Q_table: \n",np.shape(Q_table))



policy: 
 (9, 3, 2)

 Q_table: 
 (9, 3, 2)


In [11]:
def epsilon_greedy(Q_values, epsilon):    
    probs = np.full_like(Q_values, epsilon / Q_values.shape[-1], dtype=float)  # initialize with exploration probability
    best_actions = np.argmax(Q_values, axis=-1)  # find the best action for each state
    np.put_along_axis(probs, np.expand_dims(best_actions, axis=-1), 1 - epsilon + (epsilon / Q_values.shape[-1]), axis=-1)
    return probs



def train_rescorla_wagner(df, alpha, beta, Q_init=None):
    if Q_init is None:
        Q_init = Q_table.copy()
    Q_values = Q_init.copy()
    q_value_pairs = []
    choices = []
    predicted_probs = []
    distributions = []
    card_numbers = []
    
    for _, row in df.iterrows():
        action = actions[row["choice"]] 
        distribution = distributions_map[row["distribution"]] 
        card_number = row["myCard"]-1 # since I'm using this as an index! I need to do -1 to make the 1 to 9 cards come to 0 to 8
        reward = 0.5 if row["outcome"] == "win" else -0.5


        probs = epsilon_greedy(Q_values, beta)
        predicted_probs.append(probs[card_number][distribution][action])
        
        prediction_error = reward - Q_values[card_number][distribution][action]
        Q_values[card_number][distribution][action] += alpha * prediction_error
        
        q_value_pairs.append(Q_values.copy())
        choices.append(action)
        distributions.append(distribution)
        card_numbers.append(card_number)
        

    return np.array(q_value_pairs), np.array(choices), np.array(predicted_probs), np.array(distributions), np.array(card_numbers)


# this is for the sake of parallel computing
def compute_log_likelihood(alpha, beta, df_all, Q_table):
    Q_init_participant = Q_table.copy()
    q_values, choices, predicted_probs, distributions, card_numbers = train_rescorla_wagner(df_all, alpha, beta, Q_init=Q_init_participant.copy())
    
    predicted_probs = np.clip(predicted_probs, 1e-6, 1)  # prevent log(0)
    log_likelihood = np.sum(np.log(predicted_probs))
    
    return (alpha, beta, log_likelihood)


In [12]:
num_of_samples = 100
# num_of_samples = 1000
alpha_min = 0
alpha_max = 1
beta_min = 0
beta_max  = 1
alpha_samples = np.random.uniform(alpha_min, alpha_max + np.finfo(float).eps, num_of_samples)
beta_samples = np.random.uniform(beta_min, beta_max + np.finfo(float).eps, num_of_samples)

In [ ]:
BIC_models = []
AIC_models = []
best_alpha_models = []
best_beta_models = []
accuracy_models = []
precision_models = []
sensitivity_recall_models = []
specificity_models = []
f1_score_models = []
mcFadden_r2_models = []
r2_models = []

for idx, df_all in enumerate(dataframes):
    Q_init_participant = Q_table.copy()
    
    df_all = df_all[df_all['outcome'].str.lower() != 'na'].reset_index(drop=True)


    def objective(trial):
        alpha = trial.suggest_float("alpha", alpha_min, alpha_max)
        beta  = trial.suggest_float("beta", beta_min, beta_max)

        # negative log-likelihood (Optuna minimises)
        _, _, ll = compute_log_likelihood(alpha, beta,
                                        df_all,
                                        Q_init_participant.copy())
        return -ll

    study = optuna.create_study(direction="minimize",
                                sampler=optuna.samplers.TPESampler(seed=42))
    study.optimize(objective, n_trials=400, n_jobs=-1)

    best_alpha = study.best_params["alpha"]
    best_beta  = study.best_params["beta"]
    best_log_likelihood = -study.best_value

    # keep this for plotting later
    results_df = study.trials_dataframe()
    results_df["alpha"] = results_df["params_alpha"]
    results_df["beta"]  = results_df["params_beta"]
    results_df["log_likelihood"] = -results_df["value"]

    # model prediction 
    
    q_values, choices, predicted_probs, distributions, card_numbers = train_rescorla_wagner(df_all, best_alpha, best_beta, Q_init=Q_init_participant.copy())
    
    
    predicted_choices = []
    for trial in range(len(card_numbers)):
        test_action_probs = epsilon_greedy(q_values[trial], best_beta)
        p_arrowup = test_action_probs[card_numbers[trial]][distributions[trial]][actions["arrowup"]]
        p_arrow_down = test_action_probs[card_numbers[trial]][distributions[trial]][actions["arrowdown"]]
        # choosing 1 or 0 based on the softmax probabilities:
        predicted_choices.append(np.random.choice([1, 0], p=[p_arrowup, p_arrow_down]))

    # finding out model total reward based on the model's predicted choices
    total_reward = [] 
    for i in range(len(predicted_choices)):
        if len(total_reward)> 0:
            last_reward = total_reward[-1]  #  the last reward value
        else:
            last_reward = 10 # initial reward is $10
        
        if ((df_all.loc[i, 'myCard'] > df_all.loc[i, 'yourCard'] and predicted_choices[i] == 1) or
            (df_all.loc[i, 'myCard'] < df_all.loc[i, 'yourCard'] and predicted_choices[i] == 0)):
            total_reward.append(last_reward + 0.5)
        else:
            total_reward.append(last_reward - 0.5)

    
   
       # confusion matrix:
    conf_matrix = confusion_matrix(choices, predicted_choices)
    TN, FP, FN, TP = conf_matrix.ravel()  # unpacking the confusion matrix
    # acc
    accuracy = (TP + TN) / (TP + TN + FP + FN)
    # precision: From the ones that we’ve announced them as up/down, which ones are really up/down?
    precision = TP / (TP + FP) if (TP + FP) != 0 else 0
    # recall or sensitivity : true positive rate
    sensitivity_recall = TP / (TP + FN) if (TP + FN) != 0 else 0
    # specificity : true negative rate
    specificity = TN / (TN + FP) if (TN + FP) != 0 else 0
    # f1 Score
    f1_score = 2 * (precision * sensitivity_recall) / (precision + sensitivity_recall) if (precision + sensitivity_recall) != 0 else 0

    
    # bayes information criterion:
    n_trials = len(df_all)
    k = 2  # number of free parameters: alpha and beta
    BIC = k * np.log(n_trials) - 2 * best_log_likelihood # this is BIC formula based on the log lkelihode I found before

    

    # Akaike  information criterion(AIC):
    AIC = 2 * k - 2 * best_log_likelihood 


    # mcFadden r-squared:
    p_null = np.mean(choices)  # probability of choosing "1" in the dataset
    log_likelihood_null = np.sum(choices * np.log(p_null) + (1 - choices) * np.log(1 - p_null))
    mcFadden_r2 = 1 - (best_log_likelihood / log_likelihood_null)

    # r-squared
    r2 = r2_score(choices, predicted_choices)
    
    # saving models evaluation variables:
    best_alpha_models.append(best_alpha)
    best_beta_models.append(best_beta)
    BIC_models.append(BIC)
    AIC_models.append(AIC)
    accuracy_models.append(accuracy)
    precision_models.append(precision)
    sensitivity_recall_models.append(sensitivity_recall)
    specificity_models.append(specificity)
    f1_score_models.append(f1_score)
    mcFadden_r2_models.append(mcFadden_r2)
    r2_models.append(r2)


    ###########################################################################################
    ## visulization
    ###########################################################################################

    fig, axes = plt.subplots(1, 3, figsize=(19, 6))

    plots_smooth_level = 20


#############################################
    # Density Plot (KDE)
    sns.kdeplot(
        x=results_df["alpha"], 
        y=results_df["beta"], 
        fill=True, 
        cmap="viridis", 
        ax=axes[0], 
        bw_adjust=1.8,  # Increase for smoother density
        levels=plots_smooth_level,  # More contour levels
        thresh=0  # Ensure density is plotted across all values
    )
    mappable = axes[0].collections[0]
    cbar = fig.colorbar(mappable, ax=axes[0], label="density", fraction=0.046, pad=0.04)  
    cbar.ax.yaxis.set_major_formatter(mticker.FormatStrFormatter('%.2f'))  # 2 decimal places
    cbar.ax.set_ylabel("density", fontsize=12, fontweight='bold')
    cbar.ax.tick_params(labelsize=12)

    axes[0].set_xlim(alpha_min, alpha_max)
    axes[0].set_ylim(beta_min, beta_max)
    axes[0].set_xlabel("learning rate (α)", fontsize=14, fontweight='bold')
    axes[0].set_ylabel("epsilon (ε)", fontsize=14, fontweight='bold')
    axes[0].set_title("density of α and ε joint probability", fontsize=16, fontweight='bold')
    axes[0].tick_params(axis='both', labelsize=14)
#############################################


#############################################
    # Log Likelihood 

    
    alpha_step = 0.1
    beta_step = 0.1
    alpha_bins = np.arange(alpha_min, alpha_max+ alpha_step, alpha_step)  # bins from 0 to 1 with step 0.1
    beta_bins = np.arange(beta_min, beta_max + beta_step, beta_step)       # bins from 0 to 1 with step 0.1

    results_df["alpha_binned"] = pd.cut(results_df["alpha"], bins=alpha_bins, labels=alpha_bins[:-1], include_lowest=True)
    results_df["beta_binned"] = pd.cut(results_df["beta"], bins=beta_bins, labels=beta_bins[:-1], include_lowest=True)

    heatmap_data = results_df.groupby(
    ["beta_binned", "alpha_binned"], observed=False)["log_likelihood"].mean().unstack()

    heatmap_data.index = heatmap_data.index.astype(float)
    heatmap_data.columns = heatmap_data.columns.astype(float)

    sns.heatmap(
        heatmap_data, 
        cmap="Blues", 
        cbar=True,
        ax=axes[1]
    )
    axes[1].set_xticks(np.arange(len(heatmap_data.columns)))  
    axes[1].set_xticklabels([f"{x:.1f}" for x in heatmap_data.columns], rotation=45)

    axes[1].set_yticks(np.arange(len(heatmap_data.index))) 
    axes[1].set_yticklabels([f"{x:.1f}" for x in heatmap_data.index]) 


    axes[1].set_xlabel("learning rate (α)", fontsize=14, fontweight='bold')
    axes[1].set_ylabel("epsilon (ε)", fontsize=14, fontweight='bold')
    axes[1].set_title("log likelihood for combinations of α and ε", fontsize=16, fontweight='bold')
    axes[1].tick_params(axis='both', labelsize=14)
    axes[1].invert_yaxis()


    

#############################################


#############################################
    # Confusion Matrix
    heatmap_cmap_color = mcolors.LinearSegmentedColormap.from_list("warm_red", ["#fff5e6", "#ff5733"])
    sns.heatmap(
        conf_matrix, annot=True, fmt="d", cmap=heatmap_cmap_color,
        xticklabels=["arrowdown", "arrowup"], 
        yticklabels=["arrowdown", "arrowup"], 
        ax=axes[2], 
        cbar=False
    )

    axes[2].set_xlabel("prediction", fontsize=14, fontweight='bold')
    axes[2].set_ylabel("true label", fontsize=14, fontweight='bold')
    axes[2].set_title(f"confusion matrix (α={best_alpha:.2f}, ε={best_beta:.2f})", fontsize=16, fontweight='bold')
    axes[2].tick_params(axis='both', labelsize=14)


#############################################
    # saving figures
    plt.tight_layout(rect=[0, 0, 1, 0.9]) 
    fig.suptitle(f'participant {idx}', fontsize=18, fontweight='bold', y=0.95)

    filename = os.path.join(output_dir_plots, f"plot_{participants[idx]}.pdf")
    plt.savefig(filename, format='pdf')
    plt.close(fig)

    print(f"saved: {filename}")


#############################################

    # saving model behavior
    q_values_reshaped = [q_values[i].tolist() for i in range(n_trials)]  # convert each (9,3,2) array into a list format

    df_model_behavior = pd.DataFrame({
        "model_choices": predicted_choices,
        "participant_choices": choices,
        "model_total_reward": total_reward,
        "participant_total_reward": df_all["totalReward"],
        "q_val": q_values_reshaped  
    })

    file_path = os.path.join(output_dir_model_behavior, f"model_behavior_{participants[idx]}.csv")
    df_model_behavior.to_csv(file_path, index=False)



[I 2025-06-10 14:04:28,765] A new study created in memory with name: no-name-fc1e2994-4cf0-436c-80b7-c2f56a5975d7
[I 2025-06-10 14:04:30,206] Trial 6 finished with value: 183.37755054341807 and parameters: {'alpha': 0.9542582447418688, 'beta': 0.9690447252812161}. Best is trial 6 with value: 183.37755054341807.
[I 2025-06-10 14:04:30,231] Trial 0 finished with value: 189.02134035576213 and parameters: {'alpha': 0.18337240800615395, 'beta': 0.13641216149549296}. Best is trial 6 with value: 183.37755054341807.
[I 2025-06-10 14:04:30,252] Trial 3 finished with value: 156.63996744321855 and parameters: {'alpha': 0.6019635664095095, 'beta': 0.5525626494222762}. Best is trial 3 with value: 156.63996744321855.
[I 2025-06-10 14:04:30,279] Trial 8 finished with value: 156.18140257177186 and parameters: {'alpha': 0.2926920246120265, 'beta': 0.32404235829749906}. Best is trial 8 with value: 156.18140257177186.
[I 2025-06-10 14:04:30,279] Trial 5 finished with value: 165.99435826280478 and paramet

saved: 13_RL_agent_TDlearn_output\plots\plot_07_11_2024_17_23_43.pdf


[I 2025-06-10 14:04:59,271] Trial 1 finished with value: 176.22552984495104 and parameters: {'alpha': 0.19437488508699585, 'beta': 0.9263451278815003}. Best is trial 1 with value: 176.22552984495104.
[I 2025-06-10 14:04:59,501] Trial 2 finished with value: 166.53580907432652 and parameters: {'alpha': 0.6333972351445204, 'beta': 0.7457266146225078}. Best is trial 2 with value: 166.53580907432652.
[I 2025-06-10 14:05:00,104] Trial 5 finished with value: 242.7051532345139 and parameters: {'alpha': 0.3563282042086916, 'beta': 0.040106341577509586}. Best is trial 2 with value: 166.53580907432652.
[I 2025-06-10 14:05:00,130] Trial 6 finished with value: 186.5333781362669 and parameters: {'alpha': 0.6084010603328659, 'beta': 0.9945583143598818}. Best is trial 2 with value: 166.53580907432652.
[I 2025-06-10 14:05:00,159] Trial 3 finished with value: 171.95611530095908 and parameters: {'alpha': 0.6058770573403934, 'beta': 0.835889264394192}. Best is trial 2 with value: 166.53580907432652.
[I 20

saved: 13_RL_agent_TDlearn_output\plots\plot_08_11_2024_13_03_29.pdf


[I 2025-06-10 14:05:27,774] Trial 2 finished with value: 257.7788722845855 and parameters: {'alpha': 0.9370815604636865, 'beta': 0.057511989959794785}. Best is trial 0 with value: 155.79417789230243.
[I 2025-06-10 14:05:28,096] Trial 4 finished with value: 330.7286615311358 and parameters: {'alpha': 0.7404351368848182, 'beta': 0.01949597751463039}. Best is trial 0 with value: 155.79417789230243.
[I 2025-06-10 14:05:28,259] Trial 3 finished with value: 164.72205767518645 and parameters: {'alpha': 0.34650153488531055, 'beta': 0.8304489342764275}. Best is trial 0 with value: 155.79417789230243.
[I 2025-06-10 14:05:28,287] Trial 5 finished with value: 185.56314848692438 and parameters: {'alpha': 0.23093467844519744, 'beta': 0.9899997609725391}. Best is trial 0 with value: 155.79417789230243.
[I 2025-06-10 14:05:28,345] Trial 6 finished with value: 189.10149868267368 and parameters: {'alpha': 0.6584727927882255, 'beta': 0.182230845717676}. Best is trial 0 with value: 155.79417789230243.
[I 

...

saved: 13_RL_agent_TDlearn_output\plots\plot_26_11_2024_14_31_40.pdf


[I 2025-06-10 14:18:52,943] Trial 31 finished with value: 181.9422433704632 and parameters: {'alpha': 0.8332943930841828, 'beta': 0.934278140351119}. Best is trial 31 with value: 181.9422433704632.
[I 2025-06-10 14:18:52,998] Trial 11 finished with value: 185.91855770913313 and parameters: {'alpha': 0.719889815598425, 'beta': 0.4042262250039341}. Best is trial 10 with value: 175.21121655059383.
[I 2025-06-10 14:18:52,974] Trial 7 finished with value: 187.0638788620858 and parameters: {'alpha': 0.030768433756662827, 'beta': 0.9985643654752685}. Best is trial 31 with value: 181.9422433704632.
[I 2025-06-10 14:18:52,974] Trial 10 finished with value: 175.21121655059383 and parameters: {'alpha': 0.5864036718315601, 'beta': 0.8106160456963961}. Best is trial 10 with value: 175.21121655059383.
[I 2025-06-10 14:18:52,974] Trial 23 finished with value: 221.10106735089903 and parameters: {'alpha': 0.7392958096337588, 'beta': 0.221994193832883}. Best is trial 31 with value: 181.9422433704632.
[I

saved: 13_RL_agent_TDlearn_output\plots\plot_28_11_2024_12_21_16.pdf


[I 2025-06-10 14:19:17,030] Trial 17 finished with value: 169.44775946936835 and parameters: {'alpha': 0.7217065388742873, 'beta': 0.7652939175017901}. Best is trial 17 with value: 169.44775946936835.
[I 2025-06-10 14:19:17,083] Trial 2 finished with value: 174.6478627995986 and parameters: {'alpha': 0.1788886810220648, 'beta': 0.8565854297291557}. Best is trial 17 with value: 169.44775946936835.
[I 2025-06-10 14:19:17,089] Trial 14 finished with value: 179.022068039408 and parameters: {'alpha': 0.565455320188583, 'beta': 0.9141046982998936}. Best is trial 17 with value: 169.44775946936835.
[I 2025-06-10 14:19:17,104] Trial 19 finished with value: 170.5514740321493 and parameters: {'alpha': 0.5421015217287679, 'beta': 0.7882302194120081}. Best is trial 17 with value: 169.44775946936835.
[I 2025-06-10 14:19:17,196] Trial 20 finished with value: 563.9343415484256 and parameters: {'alpha': 0.012029808916497964, 'beta': 0.0292527077183673}. Best is trial 17 with value: 169.44775946936835.


saved: 13_RL_agent_TDlearn_output\plots\plot_28_11_2024_22_38_25.pdf


# now saving the model evaluation values

In [14]:
df_models_evaluation = pd.DataFrame({
    "participants": participants,
    "best_alpha": best_alpha_models,
    "best_beta": best_beta_models,
    "BIC": BIC_models,
    "AIC": AIC_models,
    "accuracy": accuracy_models,
    "precision": precision_models,
    "sensitivity_recall": sensitivity_recall_models,
    "specificity": specificity_models,
    "f1_score": f1_score_models,
    "mcFadden_r2": mcFadden_r2_models,
    "r2": r2_models
})

file_path = os.path.join(output_dir_model_evaluation, "models_evaluation.csv")
df_models_evaluation.to_csv(file_path, index=False)